# Cylindrical Nanowire Hamiltonian Spectrum
In this notebook, I will investigate the Hamiltonian of a cylindrical nanowire SNS junction. Spin-orbit coupling and Zeeman term will be assumed to be present. 

Consider a nanowire placed longitudinally along the z-axis. A magnetic field $B_z$ is also assumed to be present along the z-axis.

$$ H_e = \frac{p_z^2 + p_\phi^2 + p_r^2}{2m} - \mu + \alpha (\vec{\sigma} \times \vec{p}) \cdot \hat{r} + \frac{1}{2} g \mu_B \hbar B_z \sigma_z $$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import itertools

In [18]:
# create the Hamiltonian matrix

def calc_Hamiltonian(params):
    '''
    Returns the Hamiltonian in the real space basis.
    params is a dict with the following information:
        N_z : number of points along z
        N_phi : number of points along azimuthal direction
        t_z : h^2/(2 m a^2), where a is lattice spacing
        t_phi : h^2/(2 m R^2), where a is lattice spacing
        mu : chemical potential
        alpha : spin orbit coupling constant*h/a
        E_z : Zeeman splitting energy (0.5*g*mu_B*h*B_z) 
    '''
    N_z = params['N_z']
    N_phi = params['N_phi']
    t_z = params['t_z']
    t_phi = params['t_phi']
    mu = params['mu']
    alpha = params['alpha']
    E_z = params['E_z']
    flux = params['flux']
    
    def calc_matrix_element(x,y):
        '''
        Returns the matrix element between two real space points x and y
        '''
        (z1,phi1) = x
        (z2,phi2) = y
        # onsite element
        if z1 == z2 and phi1 == phi2:
            if N_phi != 1:
                diag_ele = 2*t_z + np.abs(t_phi)*(2 - (2*np.pi*flux/N_phi)**2) - mu
            else:
                # diagonal elemenet for N_phi = 1 does not make sense
                diag_ele = 2*t_z - mu
            return np.array([[diag_ele + E_z,0],[0,diag_ele - E_z]])
        # z hopping
        elif abs(z1-z2) == 1 and phi1 == phi2:
            return np.array([[t_z,1j*alpha],[-1j*alpha,t_z]])
        # phi hopping
        elif (phi1-phi2 == 1 or phi1-phi2 == N_phi-1)and z1 == z2:
            return np.array([[-t_phi,0],[0,np.conj(t_phi)]])
        elif (phi1-phi2 == -1 or phi1-phi2 == -N_phi+1)and z1 == z2:
            return np.conj(np.array([[-t_phi,0],[0,np.conj(t_phi)]])).T
        else:
            return np.array([[0,0],[0,0]])
    
    basis = list(itertools.product(range(N_z),range(N_phi)))
    H_list = [calc_matrix_element(x,y) for x in basis for y in basis]
    N = N_phi*N_z
    H = np.array(H_list).reshape((N,N,2,2))
    
    # magic to flatten the Hamiltonian
    # Are you wathing closely?
    H = np.array([H[x,:,y,:].flatten() for x in range(H.shape[0]) for y in range(H.shape[2])])\
    .flatten().reshape(2*N,2*N)
    
    return H
    

In [26]:
# test of the Hamiltonian function
params = {
    "N_z" : 2,
    "N_phi" : 1,
    "flux" : 0e0,
    "t_z" : 5e-3,
    "Delta" : 1e-3,
    "mu" : 0e-3,
    "alpha" : 0,
    "E_z" : 0
}

params["t_phi"] = 1e-3*np.exp(1j*2*np.pi*params["flux"]/params["N_phi"])

H = calc_Hamiltonian(params)
print(H,H.shape)

[[0.01 +0.j 0.   +0.j 0.005+0.j 0.   +0.j]
 [0.   +0.j 0.01 +0.j 0.   -0.j 0.005+0.j]
 [0.005+0.j 0.   +0.j 0.01 +0.j 0.   +0.j]
 [0.   -0.j 0.005+0.j 0.   +0.j 0.01 +0.j]] (4, 4)


# Notes
 - I am unsure how to incorporate the flux term in the diagonal element.
 
 
 Next step: Look at the spectrum!